## Pipeline-1: Setup DB + Encrypted QR Generation

In [27]:
import sqlite3, os, base64, secrets, qrcode, json
from cryptography.hazmat.primitives.ciphers.aead import AESGCM

In [28]:
KEY = os.urandom(32)  # AES-256 key; reuse same key for all QRs in this prototype
# Note : This key is to be kept secret!
print(KEY)

b'\x16\xb3z\xfc>\xfa2\xf5\xe1T\xc3/!\x95\xcbq\x07\xde\xd3;\xad\x9bJ\xe3^\x04\x0ca\xd5\x86f\x83'


In [29]:
def encrypt_token(token : str) -> str:
    aesgcm = AESGCM(KEY)
    iv = os.urandom(12) # initialization vector
    ct = aesgcm.encrypt(iv, token.encode(), b"college-qr-v1")  # ciphertext+tag
    blob = base64.urlsafe_b64encode(iv + ct).decode().rstrip("=")
    return blob

In [59]:
conn = sqlite3.connect("students.db")
cursor = conn.cursor()

# Clearing the db
cursor.execute("""
DROP TABLE IF EXISTS students;
""")
cursor.execute("""
DROP TABLE IF EXISTS logs;
""")

In [60]:
# Setting up DB - Create students table
cursor.execute("""
CREATE TABLE IF NOT EXISTS students (
    qr_token TEXT PRIMARY KEY,
    roll_no TEXT NOT NULL,
    name TEXT NOT NULL,
    department TEXT NOT NULL,
    contact_no VARCHAR(15) NOT NULL,
    email_id TEXT NOT NULL,
    aadhar_no VARCHAR(12) NOT NULL,
    dob DATE NOT NULL,
    address TEXT NOT NULL,
    bank_account_no TEXT NOT NULL,
    bank_ifsc_code TEXT NOT NULL,
    embedding1 BLOB,
    embedding2 BLOB,
    embedding3 BLOB
)
""")

# Create logs table
cursor.execute("""
CREATE TABLE logs (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    qr_token TEXT,
    roll_no TEXT,
    name TEXT,
    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
)
""")

In [ ]:
students = [
    # roll_no, name, department, contact_no, email_id, aadhar_no, dob, address, bank_account_no, bank_ifsc_code
    ("101", "Tushar Sharma", "Computer Science", "+XXXXXXXXXXXX", "tushar.sharma@uni.edu", "111122223333", "2000-05-15", "123 Main St, Anytown", "9876543210123456", "ICIC0001234"),
    ("102", "Dhairya Patel", "Electronics", "+XXXXXXXXXXXX", "dhairya.patel@uni.edu", "444455556666", "2001-11-20", "456 Oak Ave, Somewhere", "1234567890123456", "HDFC0005678"),
    ("103", "Debapriya Ghosh", "Mechanical Engineering", "+XXXXXXXXXXXX", "debapriya.ghosh@uni.edu", "777788889999", "1999-07-25", "789 Pine Ln, Elsewhere", "5432109876543210", "SBIN0009012"),
    ("104", "Archit Verma", "Civil Engineering", "+XXXXXXXXXXXX", "archit.verma@uni.edu", "000011112222", "2002-03-10", "321 Cedar Rd, Nowheres", "6789012345678901", "AXIS0003456"),
    # ("105", "Riya Singh", "Computer Science", "8008877665", "riya.singh@uni.edu", "333344445555", "2001-01-01", "10 Downing St, London", "2109876543210987", "KOTK0007890"),
    # ("106", "Sanjay Kumar", "Electronics", "7997766554", "sanjay.kumar@uni.edu", "666677778888", "2000-12-31", "55 Park Ave, New York", "8765432109876543", "YESB0001122")
]

In [62]:
def create_entry(student_tuple):
    qr_token = secrets.token_hex(16)  # random 128-bit token
    conn = sqlite3.connect("students.db")
    cursor = conn.cursor()
    cursor.execute("""
    INSERT INTO students (qr_token, roll_no, name, department, contact_no, email_id, aadhar_no, dob, address, bank_account_no, bank_ifsc_code)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (qr_token, *student_tuple))
    conn.commit()
    conn.close()

In [63]:
for student in students:
    create_entry(student)

In [64]:
def generate_qrs():
    os.makedirs("qrcodes", exist_ok=True)
    conn = sqlite3.connect("students.db")
    cursor = conn.cursor()
    cursor.execute("SELECT qr_token, roll_no FROM students")
    rows = cursor.fetchall()
    for qr_token, roll_no in rows:
        blob = encrypt_token(qr_token)   # encrypt the token
        qr = qrcode.make(blob)           # generate QR with encrypted blob
        qr.save(f"qrcodes/{roll_no}.png")

    conn.close()
    print("Encrypted QR codes saved in 'qrcodes/' folder.")

In [65]:
generate_qrs()

Encrypted QR codes saved in 'qrcodes/' folder.


In [ ]:
# # Clear logs table

# conn = sqlite3.connect("students.db")
# cursor = conn.cursor()
# cursor.execute("""
#     DELETE FROM logs
# """)
# conn.commit()
# conn.close()

## Pipeline-2 : Scanning + Decryption + Lookup

In [ ]:
# import cv2
# from pyzbar.pyzbar import decode

In [ ]:
# def decrypt_blob(blob_b64u: str) -> str:
#     # restore padding if stripped
#     blob_b64u += "=" * ((4 - len(blob_b64u) % 4) % 4)
#     raw = base64.urlsafe_b64decode(blob_b64u)
#     iv, ct = raw[:12], raw[12:]
#     aesgcm = AESGCM(KEY)
#     qr_token = aesgcm.decrypt(iv, ct, b"college-qr-v1").decode()
#     return qr_token

In [ ]:
# def get_student(qr_token):
#     conn = sqlite3.connect("students.db")
#     cursor = conn.cursor()
#     cursor.execute("SELECT roll_no, name FROM students WHERE qr_token=?", (qr_token,))
#     student = cursor.fetchone()
#     conn.close()
#     return student

In [ ]:
# try:
#     cap = cv2.VideoCapture(0)
#     if not cap.isOpened():
#         raise Exception("Could not open camera")
    
#     print("Hold the QR code in front of the camera...")
#     print("Press 'q' to quit")

#     student = None
#     found = False

#     while True:
#         ret, frame = cap.read()
#         if not ret:
#             break

#         for barcode in decode(frame):
#             try:
#                 blob = barcode.data.decode("utf-8")
#                 qr_token = decrypt_blob(blob)
#                 student = get_student(qr_token)
#                 found = True
#                 break
#             except:
#                 # Silently ignore decryption/lookup errors
#                 found = True
#                 break

#         if found:
#             break

#         try:
#             cv2.imshow("QR Scanner", frame)
#             key = cv2.waitKey(1) & 0xFF
#             if key == ord('q'):
#                 break
#         except:
#             # Silently ignore display errors
#             pass

# finally:
#     cap.release()
#     try:
#         cv2.destroyAllWindows()
#     except:
#         pass

#     if student:
#         print(f"Roll No: {student[0]}, Name: {student[1]}")
#     elif not found:
#         print("No QR detected.")
#     else:
#         print("Student not found in database.")


Hold the QR code in front of the camera...
Press 'q' to quit
Student not found in database.
